# Jedgment Summarization



## 安裝套件


In [1]:
! pip install transformers
! pip install datasets
! pip install torcheval
! pip install scikit-learn
! pip install torch
! pip install pytorch-ignite
! pip install ipywidgets
! pip install jieba

     ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
     --------------------------------------- 0.0/19.2 MB 640.0 kB/s eta 0:00:30
     --------------------------------------- 0.0/19.2 MB 640.0 kB/s eta 0:00:30
     --------------------------------------- 0.0/19.2 MB 640.0 kB/s eta 0:00:30
     --------------------------------------- 0.0/19.2 MB 178.6 kB/s eta 0:01:48
     --------------------------------------- 0.1/19.2 MB 252.2 kB/s eta 0:01:16
     --------------------------------------- 0.1/19.2 MB 385.0 kB/s eta 0:00:50
     --------------------------------------- 0.1/19.2 MB 385.0 kB/s eta 0:00:50
     --------------------------------------- 0.1/19.2 MB 312.9 kB/s eta 0:01:02
     --------------------------------------- 0.2/19.2 MB 402.6 kB/s eta 0:00:48
     --------------------------------------- 0.2/19.2 MB 402.6 kB/s eta 0:00:48
     --------------------------------------- 0.2/19.2 MB 402.6 kB/s eta 0:00:48
     --------------------------------------- 0.2

In [3]:
import transformers as T
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from ignite.metrics import Rouge
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import jieba
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## 超參數
 - 學習率 (learning rate): 1e-4
 - 訓練輪數 (epochs): 10
 - 優化器 (optimizer): AdamW
 - 批次大小 (batch size): 8
 - 評量指標 (evaluation matrics)Rouge-2

In [11]:
lr = 1e-4
epochs = 10
train_batch_size = 1
validation_batch_size = 1
test_batch_size = 1
random_seed = 42
max_length = 300

# path setting that souldn't be changed
tokenizer_path = "./saved_tokenizer"
dataset_path = "./train.xlsx"
test_dataset_path = './test.xlsx'
model_path = "./saved_models"

## 載入BART模型
 - 使用huggingface裝載模型的架構、參數和tokenizer
 - 保存在路徑./cache/中
 - 用.to(device)把模型裝載入訓練設備(GPU)
 - 使用的 tokenizer 請儲存到指定路徑

In [5]:
model = T.AutoModelForSeq2SeqLM.from_pretrained("IDEA-CCNL/Randeng-BART-139M-SUMMARY", cache_dir="./cache/").to(device)
tokenizer = T.AutoTokenizer.from_pretrained("IDEA-CCNL/Randeng-BART-139M-SUMMARY", cache_dir="./cache/", model_max_length=512)
tokenizer.save_pretrained(tokenizer_path)

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/419 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/859k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/33.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

('./saved_tokenizer\\tokenizer_config.json',
 './saved_tokenizer\\special_tokens_map.json',
 './saved_tokenizer\\spiece.model',
 './saved_tokenizer\\added_tokens.json',
 './saved_tokenizer\\tokenizer.json')

In [6]:
optimizer = AdamW(model.parameters(), lr = lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

## 資料處理
 - 使用 torch.utils.data 中的 Dataset 和 Dataloader 成批次地讀取和預處理資料

In [7]:
def get_tensor(sample):
    # 將模型的輸入和ground truth打包成Tensor
    model_inputs = tokenizer.batch_encode_plus([each["origin_context"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    model_outputs = tokenizer.batch_encode_plus([each["summary"] for each in sample], padding=True, truncation=True, return_tensors="pt")
    return model_inputs["input_ids"].to(device), model_outputs["input_ids"].to(device)

class Legal_Judgment_Dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        data_df = pd.read_excel(dataset_path)
        self.data = []
        for index, row in data_df.iterrows():
            origin_context = "summary: " + str(row['裁判原文'])
            summary = row['摘要']
            self.data.append({"origin_context": origin_context, "summary": summary})

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)


- 在範例程式中有切分成 train, valid, test，實作上可將全部資料用於 train 與 valid

In [8]:
dataset = Legal_Judgment_Dataset()
train_dataset, validation_dataset = train_test_split(dataset, test_size=0.2, random_state=random_seed)
# test_dataset, validation_dataset = train_test_split(untrain_dataset, test_size=0.5, random_state=random_seed)
summary_train = DataLoader(train_dataset, collate_fn=get_tensor, batch_size=train_batch_size, shuffle=True)
summary_validation = DataLoader(validation_dataset, collate_fn=get_tensor, batch_size=validation_batch_size, shuffle=False)

rouge = Rouge(variants=["L", 2], multiref="best")
print(f"Dataset example: \n{train_dataset[0]} \n{train_dataset[1]} \n{train_dataset[2]}")

Dataset example: 
{'origin_context': 'summary: 最高法院刑事判決一○五年度台上字第三三九七號上訴人張嘉如（原名張書平）選任辯護人黃俊嘉律師上訴人許洺瑋王祥益馮鑰仁（原名馮文軍）共同選任辯護人劉嵐律師上訴人林碧玟林巾愉（原名林碧容）共同選任辯護人鍾義律師上訴人陳沛彤邱歆茹張勝鈞（原名張依濱）林君徽陳紋慧黃景微宋珮青（原名宋淑如）凌偉婷陳昕婕（原名陳雅惠）林家瑀（原名林麗娟）張靖梅上列上訴人等因違反證券交易法案件，不服台灣高等法院高雄分院中華民國一○四年十一月二十三日第二審更審判決（一○三年度重金上更㈠字第五號，起訴案號：台灣高雄地方法院檢察署九十七年度偵字第七一○七號，九十八年度偵字第一二六三九、一五五二四、二二二二三號），提起上訴，本院判決如下：主文原判決關於張勝鈞、張嘉如、許洺瑋、王祥益、馮鑰仁、林碧玟、林巾愉、陳沛彤、邱歆茹、林君徽、陳紋慧、黃景微、宋珮青、凌偉婷、陳昕婕、林家瑀、張靖梅部分均撤銷，發回台灣高等法院高雄分院。理由一、本件原判決認定張勝鈞（原名張依濱）、張嘉如（原名張書平）、許洺瑋、王祥益、馮鑰仁（原名馮文軍）、林碧玟、林巾愉（原名林碧容）、陳沛彤、邱歆茹、林君徽、陳紋慧、黃景微、宋珮青（原名宋淑如）、凌偉婷、陳昕婕（原名陳雅惠）、林家瑀（原名林麗娟）、張靖梅（以上十七人，下稱張勝鈞等十七人）有其事實欄所載之犯行，因而撤銷第一審關於張勝鈞等十七人犯證券交易法第一百七十一條第一項第一款之證券詐偽罪（下稱證券詐偽罪）部分之判決，改判均從一重論處張勝鈞等十七人共同犯證券詐偽罪刑（皆想像競合犯證券交易法第一百七十五條第一項之非法經營證券業務罪），固非無見。二、惟查：㈠、刑事訴訟法第一百五十九條之五規定之傳聞例外，乃基於當事人進行主義中之處分主義，藉由當事人等「同意」之此一處分訴訟行為，與法院之介入審查其適當性要件，將原不得為證據之傳聞證據，賦予其證據能力。本乎程序之明確性，其第一項「經當事人於審判程序同意作為證據」者，當係指當事人意思表示無瑕疵可指之明示同意而言，以別於第二項之當事人等「知而不為異議」之默示擬制同意。當事人已明示同意作為證據之傳聞證據，並經法院審查其具備適當性之要件者，若已就該證據實施調查程序，即無許當事人再行撤回同意之理，以維訴訟程序安定性、確實性之要求。所謂「經當事人於審判程序同意作為證據」

## 驗證
驗證程式
 - 將驗證資料輸入模型，用Rouge-2評價輸出的效果
 - Rouge的使用方法參考 https://pytorch.org/ignite/generated/ignite.metrics.Rouge.html

In [9]:
def evaluate(model, dataset):
    pbar = tqdm(dataset)
    pbar.set_description(f"Evaluating")
    loss_list = []
    for inputs, targets in pbar:
        loss = model(inputs, labels=targets).loss
        loss_list.append(loss.item())
        pbar.set_postfix(loss = loss.item())
        outputs = [each.replace("<unk>", "").replace("<pad>","") for each in tokenizer.batch_decode(model.generate(inputs), max_length=max_length)]
        targets = [each.replace("<unk>", "").replace("<pad>","") for each in tokenizer.batch_decode(targets)]
        print("model generate examples:")
        print(f"output: {outputs}")
        print(f"target: {targets}")
        for out, tar in zip(outputs, targets):
            sentence = " ".join(jieba.cut(out)).split()
            ground_truth = " ".join(jieba.cut(tar)).split()
            for s in sentence:
                rouge.update(([s], [ground_truth]))
            
    return rouge.compute(), np.mean(np.array(loss_list))



## 訓練
 - 將資料成批次輸入BART模型，並獲取其損失函數數值，隨後計算梯度優化
 - tqdm用來顯示模型的訓練進度

In [12]:
for ep in range(epochs):
    model.train()
    pbar = tqdm(summary_train)
    pbar.set_description(f"Training epoch [{ep+1}/{epochs}]")
    for inputs, targets in pbar:
        optimizer.zero_grad()
        loss = model(input_ids=inputs, labels=targets).loss
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss = loss.item())
    torch.save(model, f'{model_path}/ep{ep}.mod')
    model.eval()
    validation_score, validation_loss = evaluate(model, summary_validation)
    scheduler.step(validation_loss)
    print(f"Rouge-2 score on epoch {ep}:", validation_score)

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

c:\Users\fubon\anaconda3\lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Building prefix dict from the default dictionary ...


model generate examples:
output: ['</s> 按政府公法第159第1第2定:「</s>']
target: ['政府,固不分政府基於公力行政或私行政而作成或取得,然政府已作成或取得,而以政府公法第3定之方式存在者,始法之象,是政府,然未有以政府公法第3所定之形式存在之,法未人民得求政府作成之利,政府亦其要求作成政府之。又人民申提供政府,受申除有政府公法第18所定,限制公或不予提供之情形外,即提供。又政府人民申提供、更正或充政府所之定,具行政分性,申人不服得提起予求救。</s>']


Dumping model to file cache C:\Users\fubon\AppData\Local\Temp\jieba.cache
Loading model cost 0.891 seconds.
Prefix dict has been built successfully.


...

  0%|          | 0/48 [00:00<?, ?it/s]

model generate examples:
output: ['</s> 一、按「行人象之研究,取成以</s>']
target: ['政府,固不分政府基於公力行政或私行政而作成或取得,然政府已作成或取得,而以政府公法第3定之方式存在者,始法之象,是政府,然未有以政府公法第3所定之形式存在之,法未人民得求政府作成之利,政府亦其要求作成政府之。又人民申提供政府,受申除有政府公法第18所定,限制公或不予提供之情形外,即提供。又政府人民申提供、更正或充政府所之定,具行政分性,申人不服得提起予求救。</s>']
model generate examples:
output: ['</s> 按文,倘成文以外之法,揭示法</s>']
target: ['一、依智慧案件理法第32定提起上者,除有特定外,依同法第1定,用行政法於上程序相定。又於高等行政法院判之上,非以其背法令理由,不得之,行政法第242定有明文。依同法第243第1定,判不用法或用不者,背法令;而判有同第2所列各款情形之一者,然背法令。是事人於智慧法院行政判上,如依行政法第243第1定,以判有不用法或用不理由,其上或理由有具之指摘,揭示法之或其容;若成文法以外之法,揭示法之旨趣;倘司法院解或本院之判例,揭示判解之字或其容。如以行政法第243第2所列各款情形理由,其上或理由,揭示合於各款之事。上或理由如未依此方法表明,或其所表明者上法定不合,即已智慧法院行政判之背法令有具之指摘,其上自合法。 二、上人於民103年3月17日以「御工匠」商,指定使用於商法施行第19所定商品及服分表第20之棉被密封用塑真空收袋;衣用衣服防套;藏用衣服防套;衣架;塑瓶;塑品;塑硬盒;塑筒;塑盒;塑箱;相框;框;塑衣;衣帽架;床;躺椅;壁;物;物用床;物坐商品,向被上人申,被上人查,准列第0000000商(下「系</s>']
model generate examples:
output: ['</s> 污治罪例第8第2前段定,</s>']
target: ['94年2月2日修正公布(95年7月1日施行)之刑法第10第2定:「公者,下列人:一、依法令服於家、地方自治所而具有法定限,以及其他依法令事於公共事,而具有法定限者。二、受家、地方自治所依法委,事委限有之公共事者」。上第一款段「其他依法令事於公共事,而具有法定限者」,乃理所「授公」

  0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/48 [00:00<?, ?it/s]

model generate examples:
output: ['</s> 一、按「行於之目的在於填所生之</s>']
target: ['政府,固不分政府基於公力行政或私行政而作成或取得,然政府已作成或取得,而以政府公法第3定之方式存在者,始法之象,是政府,然未有以政府公法第3所定之形式存在之,法未人民得求政府作成之利,政府亦其要求作成政府之。又人民申提供政府,受申除有政府公法第18所定,限制公或不予提供之情形外,即提供。又政府人民申提供、更正或充政府所之定,具行政分性,申人不服得提起予求救。</s>']
model generate examples:
output: ['</s> (一)按商法第243第1定:「商</s>']
target: ['一、依智慧案件理法第32定提起上者,除有特定外,依同法第1定,用行政法於上程序相定。又於高等行政法院判之上,非以其背法令理由,不得之,行政法第242定有明文。依同法第243第1定,判不用法或用不者,背法令;而判有同第2所列各款情形之一者,然背法令。是事人於智慧法院行政判上,如依行政法第243第1定,以判有不用法或用不理由,其上或理由有具之指摘,揭示法之或其容;若成文法以外之法,揭示法之旨趣;倘司法院解或本院之判例,揭示判解之字或其容。如以行政法第243第2所列各款情形理由,其上或理由,揭示合於各款之事。上或理由如未依此方法表明,或其所表明者上法定不合,即已智慧法院行政判之背法令有具之指摘,其上自合法。 二、上人於民103年3月17日以「御工匠」商,指定使用於商法施行第19所定商品及服分表第20之棉被密封用塑真空收袋;衣用衣服防套;藏用衣服防套;衣架;塑瓶;塑品;塑硬盒;塑筒;塑盒;塑箱;相框;框;塑衣;衣帽架;床;躺椅;壁;物;物用床;物坐商品,向被上人申,被上人查,准列第0000000商(下「系</s>']
model generate examples:
output: ['</s> 污治罪例第8第2前段定,</s>']
target: ['94年2月2日修正公布(95年7月1日施行)之刑法第10第2定:「公者,下列人:一、依法令服於家、地方自治所而具有法定限,以及其他依法令事於公共事,而具有法定限者。二、受家、地方自治所依法委,事委限有之公共事者」。上第一款段「其他依法令事於公共事,而具有法定限者」,乃理所「

## 測試
- 使用以下程式接跑 test data

In [14]:
def test(model, test_dataset):
    model.eval()
    pbar = tqdm(test_dataset)
    pbar.set_description(f"Testing")

    for idx, testdata in tqdm(test_dataset.iterrows(), total=test_dataset.shape[0]):
        ori_data = "summary: " + str(testdata["裁判原文"])
        inputs = tokenizer(ori_data, padding=True, truncation=True, return_tensors="pt").to(device)
        outputs_context = tokenizer.decode(model.generate(**inputs)[0], max_length=max_length)
        outputs = outputs_context.replace("<unk>", "").replace("<pad>","")
        test_dataset.loc[idx, "摘要"] = outputs
            
    return test_dataset


In [15]:
test_dataset = pd.read_excel(test_dataset_path)
test_dataset['摘要'] = None
result_data = test(model, test_dataset)
result_data.to_excel("result.xlsx", index=False)

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

c:\Users\fubon\anaconda3\lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
